**The goal here is to get rank count of words in a corpus for a given class compared to the second class.** <br>

##### TODO :

- *special characters cleaner*<br>
- *Add a stopwords remover*<br>
- *Lemmatizer and Stemmer*<br>

In [24]:
import pandas as pd 
import numpy as np
import glob
import seaborn as sns

import math
import statistics #stdev

from itertools import groupby
import join
from sklearn.utils import shuffle

In [25]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
warnings.filterwarnings('ignore')

# Read data

In [26]:
df = pd.read_csv('C:/ahamid/git_wwsssuuup/IMDB Dataset.csv')
df = df.drop_duplicates()
df.rename(columns={'sentiment' : 'label'}, inplace = True) # rename column
df = df.replace({'negative': 1, 'positive': 0})
df.sample(1)
# review

,review,label
48092,"With its few touches of surrealism, LWHTRB works as low-grade horror, but as a major follow-up statement to the original, it flounders miserably.<br /><br /> Things begin somewhat promising during the telefilm's opening credits... We see and hear several interesting shots and sounds: The Baby's black crib with the overhanging, inverted cross; the kitchen knife Rosemary carried into the Castevette's apartment and dropped in shock (the utensil is shown sticking out of the hardwood floor); and the emptiness of the Bramford itself, without tenants or furniture (voice-overs can be heard here from the previous film's dialog). Interesting too is the Easter Egg hunt the titular child participates in (the eggs and baskets are also black). Once the story gets rolling, it never really 'rolls'... And what happens to Rosemary when she boards that driverless bus, and is whisked away to God-knows-where? <br /><br />Patty Duke (a poor replacement for Mia Farrow), Ray Milland and Tina Louise (as the Southwestern Whore who raises the child, ""Adrian/Andrew"") head this almost-star cast, with Ruth Gordon reprising her ""Minnie"" role.<br /><br />Although not a total failure, this sequel-of-sorts should have been released in book form first, then maybe we all could have been a bit better informed... and not left totally in the dark. A fairly recent sequel novel ""Son of Rosemary"" (1999?) is the legitimate followup by Ira Levin himself.<br /><br />",1


# Func

In [27]:
# Définition du tokenizer
def tokenizer(review):
    liste_tokens = [] #liste de tokens à retourne
    tokens = re.split('[/ : _ . = & ? -]', review) # split de l'url et stockage dans liste intérmédiaire
    for i in tokens: # boucler sur les tokens pour supprimer les vides, ce cas là arrive quand on a dans l'url deux symboles sur lesquels on split
        if i != '':
            liste_tokens.append(i) # il faut mettre dans la nouvelle liste pour être sur que l'élément n'y est pas
    return liste_tokens


def get_top_n_words(corpus, n=None):
    """
    List the top n words in a vocabulary according to occurrence in a text corpus.
    
    get_top_n_words(["I love world", "I love the python world"]) -> 
    [('love', 2),
    ('python', 2),
    ('the', 1),
    ('world', 1)]
    """
    vec = CountVectorizer(tokenizer = tokenizer, lowercase = False).fit(corpus) # ngram_range = (2,2),
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis = 0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]


def display_side_by_side(*args,titles=cycle([''])):
    html_str=''
    for df,title in zip(args, chain(titles,cycle(['</br>'])) ):
        html_str+='<th style="text-align:center"><td style="vertical-align:top">'
        html_str+=f'<h2>{title}</h2>'
        html_str+=df.to_html().replace('table','table style="display:inline"')
        html_str+='</td></th>'
    display_html(html_str,raw=True)

<font color = Darkred>${\textbf{Get dataframe of top values for both phishing and legimate corpus}}$</font>

In [28]:
# split to two subsets of data
df_pos = df.loc[df.label == 0]
df_neg = df.loc[df.label == 1]

In [29]:
# get top n words from both dfs
pos_common_words = pd.DataFrame(get_top_n_words(list(df_pos['review'])),columns=['keyword', 'count'])
pos_common_words.reset_index(inplace = True)

neg_common_words = pd.DataFrame(get_top_n_words(list(df_neg['review'])),columns=['keyword', 'count'])
neg_common_words.reset_index(inplace = True)

In [30]:
# extract 50 first elements
top_50_pos_common_words = pos_common_words.head(50).copy()
top_50_neg_common_words = neg_common_words.head(50).copy()

In [31]:
# merge dataframe 
result_pos = pd.merge(top_50_pos_common_words, neg_common_words, how="left", on=["keyword"])
result_pos.drop(['index_x'], axis=1, inplace = True)

result_neg = pd.merge(top_50_neg_common_words, pos_common_words, how="left", on=["keyword"])
result_neg.drop(['index_x'], axis=1, inplace = True)

In [32]:
# Negative DF CREATION
top_pos_words = result_pos.rename(columns={"count_x": "positive_count", "count_y": "negative_count",
                                            "index_y": "rank_in_negative", 'keyword' : 'keyword_positive'})

top_pos_words = top_pos_words[['keyword_positive', 'positive_count', 'negative_count', 'rank_in_negative']]
top_pos_words['rank_in_negative'] = top_pos_words['rank_in_negative'] + 1
top_pos_words.index = np.arange(1, (len(top_pos_words)+1))


# positive DF CREATION
top_neg_words = result_neg.rename(columns={"count_x": "negative_count", "count_y": "positive_count",
                                            "index_y": "rank_in_positive", 'keyword' : 'keyword_negative'})

top_neg_words = top_neg_words[['keyword_negative', 'negative_count', 'positive_count', 'rank_in_positive']]
top_neg_words['rank_in_positive'] = top_neg_words['rank_in_positive'] + 1
top_neg_words.index = np.arange(1, (len(top_neg_words)+1))

In [33]:
display_side_by_side(top_pos_words.head(100), top_neg_words.head(100), titles=['positive','negative'])

,keyword_positive,positive_count,negative_count,rank_in_negative
1,the,291933,274658,1
2,and,166594,135701,3
3,a,155722,150087,2
4,of,149319,133674,4
5,to,128849,132904,5
6,is,108884,95292,6
7,in,91432,79754,7
8,that,63331,65393,9
9,I,62813,70303,8
10,it,60448,61939,11
